In [ ]:
import os
import numpy as np
import random
import tifffile as tiff

# Known artifact regions to avoid (inclusive)
artifact_exclusion = {
    "1.1_OCT_uint16_Cropped_VolumeSplit_1_Reflected_RegSeq_seqSVD_Cropped_gt":
        set(range(967, 980)),
    "1.2_OCT_uint16_Cropped_VolumeSplit_2_Reflected_RegSeq_seqSVD_Cropped_gt":
        set(range(345, 359)) | set(range(474, 486)) | set(range(918, 927)),
    "1.4_OCT_uint16_Cropped_VolumeSplit_1_Reflected_RegSeq_seqSVD_Cropped_gt": 
        set(range(520, 534)) | set(range(550, 562)) | set(range(623, 637)) |
        set(range(658, 669)) | set(range(716, 729)) | set(range(982, 996)),
    "3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_Cropped_gt": 
        set(range(244, 260)) | set(range(268, 284)) | set(range(609, 620)) |
        set(range(636, 648)) | set(range(854, 869)) | set(range(883, 896)),
    "5.3_OCT_uint16_Cropped_Reflected_VolumeSplit_1_RegSeq_seqSVD_Cropped_gt": 
        set(range(104, 114)) | set(range(442, 443)) | set(range(466, 486)) |
        set(range(500, 511)) | set(range(895, 906)),
}


def simulate_missing_bscans(volume: np.ndarray, volume_name: str, missing_fraction: float = 0.1, block_size_range=(1, 4)):
    """
    Simulate missing B-scans in an OCTA volume, avoiding predefined artifact regions.

    Args:
        volume (np.ndarray): Input 3D volume of shape (D, H, W)
        volume_name (str): Identifier used to retrieve exclusion ranges
        missing_fraction (float): Fraction of slices to remove
        block_size_range (tuple): Range of block sizes to simulate
    
    Returns:
        corrupted_volume, mask, missing_indices
    """
    D, H, W = volume.shape
    num_missing = int(D * missing_fraction)
    corrupted_volume = volume.copy()
    mask = np.zeros(D, dtype=np.uint8)
    missing_indices = set()

    available = np.ones(D, dtype=bool)
    artifact_indices = artifact_exclusion.get(volume_name, set())

    print(f"Exclusion indices for {volume_name}: {sorted(artifact_indices)}")

    while np.sum(mask) < num_missing:
        block_size = random.randint(*block_size_range)

        valid_starts = []
        for start_idx in range(D - block_size + 1):
            block = list(range(start_idx, start_idx + block_size))
            if available[start_idx:start_idx + block_size].all() and not any(idx in artifact_indices for idx in block):
                valid_starts.append(start_idx)

        if not valid_starts:
            break  # no space left

        start_idx = random.choice(valid_starts)
        block_indices = list(range(start_idx, start_idx + block_size))

        for idx in block_indices:
            corrupted_volume[idx] = 0
            mask[idx] = 1
            available[idx] = False
        missing_indices.update(block_indices)

    return corrupted_volume, mask, sorted(missing_indices)


input_path = "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.1_OCTA_Vol1_Processed_Cropped_gt.tif"
# input_path = "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.2_OCTA_Vol2_Processed_Cropped_gt.tif"
# input_path = "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.4_OCTA_Vol1_Processed_Cropped_gt.tif"
# input_path = "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/3.4_OCTA_Vol2_Processed_Cropped_gt.tif"
# input_path = "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/5.3_OCTA_Vol1_Processed_Cropped_gt.tif"


# Load volume
volume = tiff.imread(input_path)
print("Volume shape:", volume.shape)

# Derive volume name (no .tif)
volume_name = os.path.splitext(os.path.basename(input_path))[0]

# Simulate missing B-scans
corrupted_volume, mask, missing = simulate_missing_bscans(volume, volume_name, missing_fraction=0.16, block_size_range=(1, 4))

# Save corrupted and mask volumes
base_dir = os.path.dirname(input_path)
base_name = os.path.splitext(os.path.basename(input_path))[0]

# Replace "_gt" with the appropriate suffix
corrupted_path = os.path.join(base_dir, f"{base_name.replace('_gt', '_corrupted')}.tif")
mask_path = os.path.join(base_dir, f"{base_name.replace('_gt', '_mask')}.tif")

tiff.imwrite(corrupted_path, corrupted_volume.astype(np.uint16), imagej=True)
mask_volume = np.tile(mask[:, None, None], (1, volume.shape[1], volume.shape[2]))
tiff.imwrite(mask_path, mask_volume.astype(np.uint8), imagej=True)

print(f"Corrupted volume saved to: {corrupted_path}")
print(f"Mask saved to: {mask_path}")
print(f"Removed {len(missing)} B-scans at indices: {missing[:10]}...")


Volume shape: (1000, 165, 400)
Exclusion indices for 3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_Cropped_gt: [244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895]
Corrupted volume saved to: /media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_Cropped_corrupted.tif
Mask saved to: /media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_Cropped_mask.tif
Removed 161 B-scans at indices: [41, 42, 43, 44, 52, 53, 64, 65, 66, 67]...


In [ ]:
import os
import numpy as np
import tifffile as tiff

# Target height
TARGET_HEIGHT = 165

# List of input file paths
input_files = [
    # "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.1_OCTA_Vol1_Processed_Cropped_gt.tif",
    # "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.2_OCTA_Vol2_Processed_Cropped_gt.tif",
    # "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/1.4_OCTA_Vol1_Processed_Cropped_gt.tif",
    "/media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/3.4_OCTA_Vol2_Processed_Cropped_gt.tif"
]

# Pad function
def pad_volume_to_target_height(volume, target_height):
    current_height = volume.shape[1]
    diff = target_height - current_height
    if diff <= 0:
        return volume  # Already correct size or larger (shouldn’t happen)
    
    pad_top = diff // 2
    pad_bottom = diff - pad_top
    padded_volume = np.pad(volume, ((0, 0), (pad_top, pad_bottom), (0, 0)), mode='constant', constant_values=0)
    return padded_volume

# Process each file
for file_path in input_files:
    print(f"Processing: {file_path}")
    
    # Load the volume
    volume = tiff.imread(file_path)
    
    # Pad the volume
    padded_volume = pad_volume_to_target_height(volume, TARGET_HEIGHT)
    
    # Sanity check
    assert padded_volume.shape[1] == TARGET_HEIGHT, "Padding failed to reach target height"
    
    # Save output
    base, ext = os.path.splitext(file_path)
    out_path = base + f"_Height165{ext}"
    tiff.imwrite(out_path, padded_volume.astype(np.uint16))
    
    print(f"Saved padded volume to: {out_path}")


Processing: /media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_Cropped_gt.tif
Saved padded volume to: /media/admin/Expansion/Mosaic_Data_for_Ipeks_Group/OCT_Inpainting_Testing/3.4_OCT_uint16_Cropped_Reflected_VolumeSplit_2_RegSeq_seqSVD_Cropped_gt_Height165.tif
